## Imports

In [1]:
import itertools as _it
if not hasattr(_it, "izip_longest"):
    _it.izip_longest = _it.zip_longest
import tracker
if not hasattr(tracker, "RunningMeanTorch"):
    class RunningMeanTorch:
        def __init__(self, *args, **kwargs):
            pass
    tracker.RunningMeanTorch = RunningMeanTorch

In [2]:
import os, sys
sys.path.append("..")
sys.path.append("../ALAE")
print(os.getcwd())
print(os.listdir())
import deeplake,zipfile,warnings
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from ALAE.alae_ffhq_inference import load_model, encode, decode
from notebooks_utils import TensorSampler

import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import logging
import numpy as np
from matplotlib import pyplot as plt
import copy
import gc
import time
import ot as pot
import ot
from torch.serialization import safe_globals
from ALAE.model     import Model          
from ALAE.defaults  import get_cfg_defaults
from ALAE.checkpointer import Checkpointer

import torchvision.utils as vutils
from torch import nn, autograd,Tensor
from typing import Optional, Tuple, Dict
from torch_fidelity import calculate_metrics
from torch.distributions import Categorical, MultivariateNormal
from torch.distributions.mixture_same_family import MixtureSameFamily
from lib.act_func import Activations
from lib.loss_func import compute_loss,compute_loss_back,compute_loss_meanflow_v1
from lib.hjbflow import ICNN1,ICNN3,evaluation_straight,train_flow_mnist,evaluation_mnist,sample_ode_hj

/home/sfh/ICNNFlow-new
['toy_test-Copy5.ipynb', 'train_hdbm-Copy2.ipynb', 'train_hdbm-0623.ipynb', 'train_hdbm-Copy6.ipynb', '__pycache__', 'lib', '.ipynb_checkpoints', 'results', 'toy_test-Copy2.ipynb', 'ofm_transfer_mb128.png', 'test_ALAE.ipynb', '8gaussiantest.ipynb', 'src', 'toy_test-Copy3.ipynb', 'notebooks_utils.py', 'toy_test-Copy4.ipynb', 'c-reflow_benchmark.ipynb', 'backup', 'train_hdbm-idp.ipynb', 'toy_test.ipynb', '.gitignore', 'creflow_bm.ipynb', 'data', 'infer_icnn_flow.py', 'train_hdbm-Copy4.ipynb', 'train_hdbm-Copy1.ipynb', 'icnn_flow.py', 'reflow_benchmark-Copy1.ipynb', 'reflow_benchmark.ipynb', 'ALAE', 'W2_images.ipynb', 'train_ALAE.ipynb', 'ofmsrc', 'train_hdbm-Copy5.ipynb', 'toy_test-Copy1.ipynb', 'train_mnist.ipynb', 'ALAE_AC.ipynb', 'notebook', 'generators2d.py', 'datasets', 'benchmarks', 'train_hdbm.ipynb', 'reflow_benchmark-Copy2.ipynb', 'largetest.ipynb']


In [3]:
log_dir = "./results/FFHQ"
os.makedirs(log_dir, exist_ok=True)
logging.basicConfig(
    filename=os.path.join(log_dir, "train.log"),
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(message)s"
)

## Parameters

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0) 

input_data = 'MAN'
target_data = 'WOMAN'

# paras
num_samples    = 100000  
iterations     = 100000
batch_size     = 1024
lr             = 1e-2
num_layers     = 10
dim            = 512
dimh           = 4096
eps            = 1e-2
act_fn         = Activations.get_act('celu')
log_interval   = 5000
loss_func=compute_loss_back

logging.info(f"hyperparams: samples={num_samples}, iters={iterations}, bs={batch_size}, lr={lr}")

## Data

In [5]:
import gdown
import os

data_dir = 'data'
if not os.path.isdir(data_dir):
    os.makedirs(data_dir)

urls = {
    "age.npy": "https://drive.google.com/uc?id=1Vi6NzxCsS23GBNq48E-97Z9UuIuNaxPJ",
    "gender.npy": "https://drive.google.com/uc?id=1SEdsmQGL3mOok1CPTBEfc_O1750fGRtf",
    "latents.npy": "https://drive.google.com/uc?id=1ENhiTRsHtSjIjoRu1xYprcpNd8M9aVu8",
    "test_images.npy": "https://drive.google.com/uc?id=1SjBWWlPjq-dxX4kxzW-Zn3iUR3po8Z0i",
}
for filename, url in urls.items():
    file_path = os.path.join(data_dir, filename)
    if os.path.exists(file_path):
        print(f"✔ {filename} exist!")
    else:
        gdown.download(url, file_path, quiet=False)

    

✔ age.npy exist!
✔ gender.npy exist!
✔ latents.npy exist!
✔ test_images.npy exist!


In [6]:
train_size = 60000
test_size = 10000

latents = np.load("data/latents.npy")#70000*512
gender = np.load("data/gender.npy")#(70000, )
age = np.load("data/age.npy") #1
test_inp_images = np.load("data/test_images.npy") #300*1024*1024*3


train_latents, test_latents = latents[:train_size], latents[train_size:]
train_gender, test_gender = gender[:train_size], gender[train_size:]
train_age, test_age = age[:train_size], age[train_size:]
test_gender300 = gender[train_size : train_size + test_inp_images.shape[0]] 
if input_data == "MAN":
    x_inds_train = np.arange(train_size)[(train_gender == "male").reshape(-1)]
    x_inds_test = np.arange(test_size)[(test_gender == "male").reshape(-1)]
elif input_data == "WOMAN":
    x_inds_train = np.arange(train_size)[(train_gender == "female").reshape(-1)]
    x_inds_test = np.arange(test_size)[(test_gender == "female").reshape(-1)]
elif input_data == "ADULT":
    x_inds_train = np.arange(train_size)[
        (train_age >= 18).reshape(-1)*(train_age != -1).reshape(-1)
    ]
    x_inds_test = np.arange(test_size)[
        (test_age >= 18).reshape(-1)*(test_age != -1).reshape(-1)
    ]
elif input_data == "CHILDREN":
    x_inds_train = np.arange(train_size)[
        (train_age < 18).reshape(-1)*(train_age != -1).reshape(-1)
    ]
    x_inds_test = np.arange(test_size)[
        (test_age < 18).reshape(-1)*(test_age != -1).reshape(-1)
    ]
x_data_train = train_latents[x_inds_train]
x_data_test = test_latents[x_inds_test]

if target_data == "MAN":
    y_inds_train = np.arange(train_size)[(train_gender == "male").reshape(-1)]
    y_inds_test = np.arange(test_size)[(test_gender == "male").reshape(-1)]
elif target_data == "WOMAN":
    y_inds_train = np.arange(train_size)[(train_gender == "female").reshape(-1)]
    y_inds_test = np.arange(test_size)[(test_gender == "female").reshape(-1)]
elif target_data == "ADULT":
    y_inds_train = np.arange(train_size)[
        (train_age >= 18).reshape(-1)*(train_age != -1).reshape(-1)
    ]
    y_inds_test = np.arange(test_size)[
        (test_age >= 18).reshape(-1)*(test_age != -1).reshape(-1)
    ]
elif target_data == "CHILDREN":
    y_inds_train = np.arange(train_size)[
        (train_age < 18).reshape(-1)*(train_age != -1).reshape(-1)
    ]
    y_inds_test = np.arange(test_size)[
        (test_age < 18).reshape(-1)*(test_age != -1).reshape(-1)
    ]
y_data_train = train_latents[y_inds_train]
y_data_test = test_latents[y_inds_test]

X_train = torch.tensor(x_data_train)
Y_train = torch.tensor(y_data_train)

X_test = torch.tensor(x_data_test)
Y_test = torch.tensor(y_data_test)

X_sampler = TensorSampler(X_train, device="cpu")
Y_sampler = TensorSampler(Y_train, device="cpu")
# print(latents.shape)
# print(gender.shape)
print(test_gender300.shape)
# print(age.shape)
# print(test_inp_images.shape)
# print(x_inds_test.shape)
# print(y_inds_test.shape)


from ALAE.discrete_ot import OTPlanSampler

def get_discrete_ot_plan_sample_fn(sampler_x, sampler_y, device='cuda'):
    ot_plan_sampler = OTPlanSampler('exact')
    def ret_fn(batch_size):
        x_samples = sampler_x.sample(batch_size).to(device)
        y_samples = sampler_y.sample(batch_size).to(device)
        return ot_plan_sampler.sample_plan(x_samples, y_samples)
    return ret_fn

sampling_fn = get_discrete_ot_plan_sample_fn(X_sampler, Y_sampler, device=device)
X_sampler, Y_sampler = sampling_fn(batch_size)
X_sampler = TensorSampler(X_sampler.detach().cpu(), device=device)
Y_sampler = TensorSampler(Y_sampler.detach().cpu(), device=device)

(300,)


## Model

In [7]:
model = ICNN3(
    dim=dim,
    dim_t=1,
    dimh=dimh,
    num_hidden_layers=num_layers,
    act_fn=act_fn,
    batch_size=batch_size,
    eps=eps
).to(device)
# model = ICNN1(
#     dim=dim,
#     dim_t=1,
#     dimh=dimh,
#     num_hidden_layers=num_layers,
#     act_fn=act_fn,
#     batch_size=batch_size,
# ).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
start_time = time.time()


## Train

In [8]:

def safe_torch_load(path, map_location="cpu"):
    if (not os.path.exists(path)) or (os.path.getsize(path) == 0) or (not zipfile.is_zipfile(path)):
        warnings.warn(f"[resume] invalid checkpoint: {path}; start fresh or fallback.")
        return None
    try:
        return torch.load(path, map_location=map_location, weights_only=True)
    except Exception as e:
        warnings.warn(f"[resume] failed to load {path}: {e}; start fresh or fallback.")
        return None

def atomic_torch_save(obj, path):
    tmp = path + ".tmp"
    with open(tmp, "wb") as f:
        torch.save(obj, f)
        f.flush(); os.fsync(f.fileno())  
    os.replace(tmp, path)  


In [ ]:
chunk = 5000
os.makedirs(log_dir, exist_ok=True)
checkpoint_path = os.path.join(log_dir, "checkpoint.pth")
checkpoint_prev = os.path.join(log_dir, "checkpoint_prev.pth")  # 

start_iter = 0
loss_history = []

ck = safe_torch_load(checkpoint_path, map_location=device)
if ck is None and os.path.exists(checkpoint_prev):
    ck = safe_torch_load(checkpoint_prev, map_location=device)

if ck is not None:
    model.load_state_dict(ck["model_state"], strict=False)
    optimizer.load_state_dict(ck["optimizer_state"])
    start_iter = ck.get("iteration", 0) + 1
    loss_history = ck.get("loss_history", [])
    print(f"from iter {start_iter} continuing, loss_history:{len(loss_history)}")
else:
    print("Begin training")


current_iter = start_iter
try:
    while current_iter < iterations:
        this_chunk = min(chunk, iterations - current_iter)
        print(f"Training iter {current_iter} to {current_iter + this_chunk - 1}")

        loss_seg = train_flow_mnist(
            model, optimizer,
            X_sampler, Y_sampler,
            iterations=this_chunk,
            batch_size=batch_size,
            loss_func=loss_func,
            log_interval=log_interval,
            benchmark=None,
            minibatch=True,
        )

        loss_history.extend(loss_seg)
        current_iter += this_chunk

        payload = {
            "iteration"      : current_iter - 1,
            "model_state"    : model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
            "loss_history"   : loss_history,
            "hyperparams"    : {
                "num_hidden_layers": num_layers,
                "dimh"             : dimh,
                "activation"       : act_fn.__name__,
                "batch_size"       : batch_size,
                "lr"               : lr,
            },
            "model_name": model.__class__.__name__,
            "model_structure": str(model),
        }

        
        if os.path.exists(checkpoint_path):
            try:
                os.replace(checkpoint_path, checkpoint_prev)
            except Exception:
                pass

        atomic_torch_save(payload, checkpoint_path)
        print(f"save checkpoint to iter {current_iter - 1}")

    print("Finish Loading")
except KeyboardInterrupt:
    payload = {
        "iteration"      : current_iter - 1,
        "model_state"    : model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "loss_history"   : loss_history,
    }
    atomic_torch_save(payload, checkpoint_path)
    print(f"Interrupted, save checkpoint to iter {current_iter - 1}")
    raise

train_time = time.time() - start_time
logging.info(f"Training completed in {train_time:.2f} seconds")


plt.figure()
plt.plot(loss_history)
plt.title("Training Loss on FFHQ")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.tight_layout()
plt.savefig(os.path.join(log_dir, "loss_FFHQ.png"))
plt.close()

/tmp/ipykernel_3175719/3520829162.py:3: UserWarning: [resume] invalid checkpoint: ./results/FFHQ/1023nosp/checkpoint0915.pth; start fresh or fallback.
  warnings.warn(f"[resume] invalid checkpoint: {path}; start fresh or fallback.")


Begin training
Training iter 0 to 4999


  0%|                                                                      | 1/5000 [00:01<2:01:08,  1.45s/it]

Iter 0/5000 — total 0.5596, FM 0.5596, HJ 0.0000


  1%|▊                                                                    | 61/5000 [01:23<1:54:36,  1.39s/it]